# Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:
| **`Tracking server`**   | yes local server |
| :---------------------: | :--------------: | 
| **`Backend store`**     | sqlite database  |
| **`Artifact store`**    | local filesystem |

The experiment can be explored locally by accessing the local tracing server.

To run this example you need to launch the `mlflow` server locally by running the following command in your terminal:

```shell
mlflow server --backend-store-uri sqlite:///backend.db
```

You can also add the parameter `--default-artifact-root <path>`

In [1]:
import mlflow

# Connect after running the following command:
# mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking-URI: '{mlflow.get_tracking_uri()}'")

tracking-URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/user/GITs/Zoomcamps/MLOps/mlops-material/2-Experiment-Tracking/artifacts_local/0', creation_time=1748028881146, experiment_id='0', last_update_time=1748028881146, lifecycle_stage='active', name='Default', tags={}>]

**Code for training a Logistic Regression Model**

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():
    X, y = load_iris(return_X_y=True)
    params = { "C": 0.1, "random_state": 42 }
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")


2025/05/23 21:35:35 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/05/23 21:35:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/home/user/GITs/Zoomcamps/MLOps/mlops-material/2-Experiment-Tracking/artifacts_local/1/14198976ee8c4b6c8e04ba6fc7d1c8ce/artifacts'
🏃 View run delicate-steed-367 at: http://127.0.0.1:5000/#/experiments/1/runs/14198976ee8c4b6c8e04ba6fc7d1c8ce
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Metadata is now stored in the SQLite database instead of the local file system.

In [7]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/user/GITs/Zoomcamps/MLOps/mlops-material/2-Experiment-Tracking/artifacts_local/1', creation_time=1748028935565, experiment_id='1', last_update_time=1748028935565, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/home/user/GITs/Zoomcamps/MLOps/mlops-material/2-Experiment-Tracking/artifacts_local/0', creation_time=1748028881146, experiment_id='0', last_update_time=1748028881146, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the Model Registry

In [8]:
from mlflow.tracking import MlflowClient
client = MlflowClient("http://127.0.0.1:5000")

In [9]:
# There are currently no registered models
client.search_registered_models()

[]

In [10]:
# Register a model
run_id = client.search_runs(experiment_ids="1")[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models", 
    name="iris-classifier"
)

Successfully registered model 'iris-classifier'.
2025/05/23 21:37:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1748029048741, current_stage='None', description='', last_updated_timestamp=1748029048741, name='iris-classifier', run_id='14198976ee8c4b6c8e04ba6fc7d1c8ce', run_link='', source='/home/user/GITs/Zoomcamps/MLOps/mlops-material/2-Experiment-Tracking/artifacts_local/1/14198976ee8c4b6c8e04ba6fc7d1c8ce/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>

Now look into the MLflow model registry. The model will be there.

In [13]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1748029048682, description='', last_updated_timestamp=1748029048741, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748029048741, current_stage='None', description='', last_updated_timestamp=1748029048741, name='iris-classifier', run_id='14198976ee8c4b6c8e04ba6fc7d1c8ce', run_link='', source='/home/user/GITs/Zoomcamps/MLOps/mlops-material/2-Experiment-Tracking/artifacts_local/1/14198976ee8c4b6c8e04ba6fc7d1c8ce/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]